In [1]:
import sys
sys.path.append('..')

from src.data import make_dataset
# from src.simulations import run_simulations
from tqdm import tqdm

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [2]:
import numpy as np
import pandas as pd

In [3]:
word_list = []
with open("..\\data\\raw\\glove.6B.50d.txt", encoding="utf8") as f:
            for line in tqdm(f, desc="Reading file", unit="line", total=400_000):
                word, coefs = line.split(maxsplit=1)
                word_list += [word]

Reading file: 100%|██████████| 400000/400000 [00:00<00:00, 469047.93line/s]


In [4]:
print("len(words_array) (raw):\t\t\t" + "{:,}".format(len(word_list)))
word_list = make_dataset.filter_alphabetic(word_list)
print("len(words_array) (filter_alphabetic):\t" + "{:,}".format(len(word_list)))
word_list = make_dataset.filter_20k(word_list, "..\\data\\external\\20k.txt")
print("len(words_array) (filter_20k):\t\t" + "{:,}".format(len(word_list)))

len(words_array) (raw):			400,000
len(words_array) (filter_alphabetic):	317,756
len(words_array) (filter_20k):		19,737


In [50]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].lower()
    if tag in "nvars": return tag
    else: return "n"

def lemmatize_words(words):
    lemmatized_set = set()
    wnl = WordNetLemmatizer()
    [lemmatized_set.add(wnl.lemmatize(word, pos=get_wordnet_pos(word))) for word in words]
    return list(lemmatized_set)

In [51]:
lemma_list = lemmatize_words(word_list)
print(len(lemma_list))
np.random.choice(lemma_list, 20)

15018


array(['statement', 'bet', 'zh', 'spectacular', 'dental', 'performs',
       'privacy', 'ietf', 'showcase', 'contextual', 'np', 'duplicate',
       'nora', 'gently', 'coke', 'devel', 'wager', 'rocker', 'japan',
       'budgetary'], dtype='<U17')

In [52]:
test_word = "performs"
print(nltk.pos_tag([test_word])[0][1][0].lower())
print(get_wordnet_pos(test_word))

n
n


In [53]:
wnl = WordNetLemmatizer()
wnl.lemmatize(test_word, pos="n")

'performs'

In [3]:
embeddings_index = make_dataset.parse_glove_data("..\\data\\raw\\glove.6B.50d.txt", True)

Reading file:   0%|          | 0/400000 [00:00<?, ?line/s]

Reading file:  20%|█▉        | 79837/400000 [00:03<00:15, 20728.04line/s]

: 

In [5]:
words_array = make_dataset.get_words_array(embeddings_index)
print("len(words_array) (raw):\t\t\t" + "{:,}".format(len(words_array)))
words_array = make_dataset.filter_alphabetic(words_array)
print("len(words_array) (filter_alphabetic):\t" + "{:,}".format(len(words_array)))
words_array = make_dataset.filter_20k(words_array, "..\\data\\external\\20k.txt")
print("len(words_array) (filter_20k):\t\t" + "{:,}".format(len(words_array)))


len(words_array) (raw):			400,000
len(words_array) (filter_alphabetic):	317,756
len(words_array) (filter_20k):		19,737


In [6]:
glove_df = make_dataset.embeddings_to_dataframe(embeddings_index, words_array)